In [1]:
import pandas as pd
import numpy as np

In [3]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

fatal: destination path 'IMDB-Movie-Reviews-Large-Dataset-50k' already exists and is not an empty directory.


In [9]:
df = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx')


In [10]:
pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx')

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report



In [11]:
df.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [12]:
! pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-as8vjjl1
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-as8vjjl1
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 96bf02872d9756f29d6cddb8aafaedcd2a39bbb4
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7602 sha256=00a8eeb3b6e0603fbaf784b7a59acc1aa6983bcc27198c11a3f328e29c48959b
  Stored in directory: /tmp/pip-ephem-wheel-cache-9sf47ift/wheels/5c/94/34/99d5ff65e88b8d9a6c5e8d8652f2311d87790a61a1b7466e21
Successfully built preprocess-kgptalkie


In [14]:
import preprocess_kgptalkie as ps
import re
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x

In [17]:
df['Reviews'] = df['Reviews'].apply(lambda x: get_clean(x))

In [18]:
df.head()

,Reviews,Sentiment
0,when i first tuned in on this morning news i t...,neg
1,mere thoughts of going overboard aka babes aho...,neg
2,why does this movie fall well below standards ...,neg
3,wow and i thought that any steven segal movie ...,neg
4,the story is seen before but that doesand matt...,neg


In [20]:
tfidf = TfidfVectorizer(max_features=5000)
X = df['Reviews']
Y = df['Sentiment']

X = tfidf.fit_transform(X)


In [21]:
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845804 stored elements in Compressed Sparse Row format>

In [25]:
X_train, X_test, Y_train, Y_test =  train_test_split(X,Y, test_size = 0.2, random_state = 0)

In [ ]:
clf = LinearSVC()
clf.fit(X_train, Y_train)


In [27]:
Y_pred = clf.predict(X_test)

In [29]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [30]:
X = 'this movie is really good. thanks a lot for making it'

X = get_clean(X)
vec = tfidf.transform([X])

In [31]:
vec.shape

(1, 5000)

In [32]:
clf.predict(vec)

array(['pos'], dtype=object)

In [33]:
import pickle

In [34]:
pickle.dump(clf, open('model','wb'))